In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.__version__

'1.12.1+cu113'

In [ ]:
import spacy

In [ ]:
spacy.__version__

'3.4.1'

In [ ]:
spacy.require_gpu()

True

In [ ]:
torch.cuda.current_device()

0

In [ ]:
import logging
import pickle
import random
import sys
from pathlib import Path
import json
import time

import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example


In [ ]:
logging.basicConfig(level=logging.INFO)

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
root.addHandler(handler)

In [ ]:
def save_model(model, output_dir):
    """
    Saves model to disk for later use.
    :param exp_name: Name of the running experiment. This is used as folder name for storing the model.
    :param model: Model being saved
    :param output_dir: Directory where to save the model.
    :return: Full path to saved model.
    """
    exp_name="clf_model_1"
    saved_model_path = Path(output_dir, exp_name)
    if not saved_model_path.exists():
        saved_model_path.mkdir(parents=True)
    model.to_disk(saved_model_path)
    logging.info("Saved model to {}".format(output_dir))
    return saved_model_path

In [ ]:
def load_or_create_empty_model(model=None):
    """
    Loads a given model or creates a blank english model.
    :param model: Optional Model to load.
    :return: Loaded or blank model.
    """
    if model:
        nlp = spacy.load(model)
        logging.debug("Loaded model {}".format(model))
    else:
        nlp = spacy.blank("en")
        logging.debug("Created blank 'en' model")
    return nlp

In [ ]:
# from spacy.tokenizer import Tokenizer
# from spacy.util import compile_prefix_regex, compile_suffix_regex
import re
import time

In [ ]:
# def custom_tokenizer(nlp):
#     infix_re = re.compile(r'''[\,\?\;\‘\’\`\“\”\"\'\[\]\(\)\<\>\~]''')
#     prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
#     suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)
#     return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 token_match=None)

In [ ]:
def _train(output_dir, n_iter, train_file,labels):
    """Load the model, set up the pipeline and train the entity recognizer."""
    

    nlp = spacy.blank("en")
    model=None
#     nlp = load_or_create_empty_model(model)
#     nlp.tokenizer=custom_tokenizer(nlp) 

    if "textcat" not in nlp.pipe_names:
        # textcat = nlp.create_pipe("textcat_multilabel")
        textcat = nlp.add_pipe("textcat", last=True)
    else:
        textcat = nlp.get_pipe("textcat")

    with open(train_file, 'r') as f:
        data = json.load(f)
        train_data = data
        # train_data = data["train"]

    ### this code can be optimized by passing all possible labels in separate list.
    train_data=train_data[:]
    for label in labels:
        textcat.add_label(label)
    nlp.initialize()
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model

        optimizer = nlp.begin_training()
        # optimizer = nlp.create_optimizer()
        print("Training started")
        for itn in range(n_iter):
            start_time = time.time()
            random.shuffle(train_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            # spacy.util.minibatch(train_data, size=1000)
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                    texts, annotations = zip(*batch)
                    example = []
                    for i in range(len(texts)):
                        doc = nlp.make_doc(texts[i])
                        example.append(Example.from_dict(doc, annotations[i]))

                    nlp.update(example,sgd=optimizer, drop=0.5, losses=losses,)


            print(itn," - Completed",time.time()-start_time)

    saved_model_path = save_model(nlp, output_dir)
    return nlp


In [ ]:
# model='en_core_web_sm'
# model=None
output_dir="/content/drive/Shareddrives/Data Discovery/temp/"
train_file="/content/drive/Shareddrives/Data Discovery/data_folder/dd_classification_train_Data.json"
# train_file="/content/drive/Shareddrives/Data Discovery/data_folder/bbc_data.json"
n_iter=30

In [ ]:
# labels = ['athletics', 'cricket', 'football', 'rugby', 'tennis']
labels=['Address', 'City', 'Country', 'County', 'Nationality', 'Organization', 'Person Name', 'Political Preference', 'Race', 'Religion', 'State', 'Street', 'Sexual orientation', 'O']


In [ ]:
start = time.perf_counter()
model= _train(output_dir, n_iter, train_file,labels)
end = time.perf_counter()
print(f"finished in {round(end-start, 2)} second(s)")

[2022-08-17 06:38:27,616] [INFO] Created vocabulary
INFO:spacy:Created vocabulary


Created vocabulary


[2022-08-17 06:38:27,627] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object


Finished initializing nlp object


[2022-08-17 06:38:27,675] [INFO] Created vocabulary
INFO:spacy:Created vocabulary


Created vocabulary


[2022-08-17 06:38:27,685] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object


Finished initializing nlp object
Training started
0  - Completed 2208.3919558525085


In [ ]:
train_file

'/content/drive/Shareddrives/Data Discovery/data_folder/dd_classification_train_Data.json'

In [ ]:
test_file='/content/drive/Shareddrives/Data Discovery/data_folder/dd_classification_test_Data.json'

In [ ]:
with open(test_file,"r") as f:
  data = json.load(f)

In [ ]:
# test = data["test"]
test = data

In [ ]:
len(test)

328191

In [ ]:
key = 23

In [ ]:
text = test[key][0]

In [ ]:
text

'kurt benfield'

In [ ]:
# model.begin_training

In [ ]:
doc = model(test[key][0])

In [ ]:
doc.cats

{'Address': 5.2939218980752045e-11,
 'City': 5.2939218980752045e-11,
 'Country': 5.2939218980752045e-11,
 'County': 5.2939218980752045e-11,
 'Nationality': 5.2939218980752045e-11,
 'Organization': 5.2939218980752045e-11,
 'Person Name': 1.0,
 'Political Preference': 5.2939218980752045e-11,
 'Race': 5.2939218980752045e-11,
 'Religion': 5.2939218980752045e-11,
 'State': 5.2939218980752045e-11,
 'Street': 5.2939218980752045e-11,
 'Sexual orientation': 5.2939218980752045e-11,
 'O': 5.2939218980752045e-11}

In [ ]:
test[key][1]

'Person Name'

In [ ]:
from spacy.scorer import Scorer

In [ ]:
labels_map = dict(zip(labels,[False]*len(labels)))

In [ ]:
test_examples = []
for d in test:
  doc = model.make_doc(d[0])
  l = labels_map.copy()
  l[d[1]]=True
  test_examples.append(Example.from_dict(doc,{"cats":l}))

In [ ]:
scorer = Scorer()

In [ ]:
result = scorer.score(test_examples)

ValueError: ignored

In [ ]:
test_examples[0].reference.cats

{'athletics': True,
 'cricket': False,
 'football': False,
 'rugby': False,
 'tennis': False}

In [ ]:
nlp = nlp = spacy.load("en_core_web_sm")

In [ ]:
textcat = nlp.add_pipe("textcat_multilabel")

In [ ]:
textcat.add_label

In [ ]:
doc = nlp.make_doc("Text is good")

In [ ]:
e = []
for _ in range(100):
  e.append(Example.from_dict(doc,{"cats":{"P":True,"F":False}}))

In [ ]:
len(e)

100

In [ ]:
batches = minibatch(e, size=compounding(4.0, 32.0, 1.001))

In [ ]:
for batch in  batches:
  break

In [ ]:
len(batch)

4

In [ ]:
batch

[{'doc_annotation': {'cats': {'P': True, 'F': False}, 'entities': ['O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['Text', 'is', 'good'], 'SPACY': [True, True, False], 'TAG': ['', '', ''], 'LEMMA': ['', '', ''], 'POS': ['', '', ''], 'MORPH': ['', '', ''], 'HEAD': [0, 1, 2], 'DEP': ['', '', ''], 'SENT_START': [1, 0, 0]}},
 {'doc_annotation': {'cats': {'P': True, 'F': False}, 'entities': ['O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['Text', 'is', 'good'], 'SPACY': [True, True, False], 'TAG': ['', '', ''], 'LEMMA': ['', '', ''], 'POS': ['', '', ''], 'MORPH': ['', '', ''], 'HEAD': [0, 1, 2], 'DEP': ['', '', ''], 'SENT_START': [1, 0, 0]}},
 {'doc_annotation': {'cats': {'P': True, 'F': False}, 'entities': ['O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['Text', 'is', 'good'], 'SPACY': [True, True, False], 'TAG': ['', '', ''], 'LEMMA': ['', '', ''], 'POS': ['', '', ''], 'MORPH': ['', '', ''], 'HEAD': [0, 1, 2], 'DEP': ['', '', ''], 'SENT_START': [1, 0, 0

In [ ]:
type(batch)

list

In [ ]:
18630.34/3600

5.175094444444444

In [ ]:
13160.99seconds->5epochs

In [ ]:
2676/60


44.6

In [ ]:
import spacy

In [ ]:
10*44

440

In [ ]:
440/60

7.333333333333333

In [ ]:
nlp=spacy.load("model_5")

/root/anaconda3/lib/python3.8/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.2.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
doc=nlp("india")

In [ ]:
for i in doc.ents:
    print(i.text,i.label_)

india Country


In [ ]:
filepath = "/content/drive/Shareddrives/Data Discovery/\
DD_StructuredData_TrainTestData&Notebooks/final_train_data.json"

In [ ]:
with open(filepath,"r") as f:
  data = json.load(f)

In [ ]:
len(data)

1329027

In [ ]:
p = data[1:8]

In [ ]:
spacy.util.minibatch(p, size=3)

<generator object minibatch at 0x7fa04ba806d0>

In [ ]:
batches = minibatch(p,size=compounding(4,32,1.002))

In [ ]:
for batch in batches:
  break

In [ ]:
batch

[['cadence rogers', {'entities': [[0, 14, 'Person Name']]}],
 ['aalderink abel', {'entities': [[0, 14, 'Person Name']]}],
 ['david christensen', {'entities': [[0, 17, 'Person Name']]}],
 ['sommerfeld lidia', {'entities': [[0, 16, 'Person Name']]}]]

In [ ]:
texts, annotations = zip(*batch)

In [ ]:
texts

('cadence rogers', 'aalderink abel', 'david christensen', 'sommerfeld lidia')

In [ ]:
annotations

({'entities': [[0, 14, 'Person Name']]},
 {'entities': [[0, 14, 'Person Name']]},
 {'entities': [[0, 17, 'Person Name']]},
 {'entities': [[0, 16, 'Person Name']]})

In [ ]:
nlp = spacy.blank("en")

In [ ]:
example=[]
for text,ent in zip(texts,annotations):
  doc = nlp.make_doc(text)
  example.append(Example.from_dict(doc,ent))

In [ ]:
example

[{'doc_annotation': {'cats': {}, 'entities': ['B-Person Name', 'L-Person Name'], 'links': {}}, 'token_annotation': {'ORTH': ['cadence', 'rogers'], 'SPACY': [True, False], 'TAG': ['', ''], 'LEMMA': ['', ''], 'POS': ['', ''], 'MORPH': ['', ''], 'HEAD': [0, 1], 'DEP': ['', ''], 'SENT_START': [1, 0]}},
 {'doc_annotation': {'cats': {}, 'entities': ['B-Person Name', 'L-Person Name'], 'links': {}}, 'token_annotation': {'ORTH': ['aalderink', 'abel'], 'SPACY': [True, False], 'TAG': ['', ''], 'LEMMA': ['', ''], 'POS': ['', ''], 'MORPH': ['', ''], 'HEAD': [0, 1], 'DEP': ['', ''], 'SENT_START': [1, 0]}},
 {'doc_annotation': {'cats': {}, 'entities': ['B-Person Name', 'L-Person Name'], 'links': {}}, 'token_annotation': {'ORTH': ['david', 'christensen'], 'SPACY': [True, False], 'TAG': ['', ''], 'LEMMA': ['', ''], 'POS': ['', ''], 'MORPH': ['', ''], 'HEAD': [0, 1], 'DEP': ['', ''], 'SENT_START': [1, 0]}},
 {'doc_annotation': {'cats': {}, 'entities': ['B-Person Name', 'L-Person Name'], 'links': {}}, 't

In [ ]:
l={}

In [ ]:
nlp.begin_training()

[2022-08-05 11:20:38,461] [INFO] Created vocabulary
INFO:spacy:Created vocabulary


Created vocabulary


[2022-08-05 11:20:38,475] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object


Finished initializing nlp object


In [ ]:
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner", last=True)

In [ ]:
ner.add_label("Person Name")
# ner.add_label("Person Name")

1

In [ ]:
nlp.update(example,drop=0.5,losses=l)

KeyError: ignored

In [ ]:
l

{}